In [5]:
import warnings, csv, os, sys, json
import pandas as pd
import json_repair
from typing import List
warnings.filterwarnings("ignore")
from langchain_openai import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    ChatPromptTemplate
)
from langchain_community.utils.openai_functions import (
    convert_pydantic_to_openai_function,
)
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

In [6]:
#variables

model_name = 'gpt-4-1106-preview'  #gpt model name


In [10]:
# This block of code is not needed, it is there to know how function calling works.

# To learn more about function calling, check this out https://www.datacamp.com/tutorial/open-ai-function-calling-tutorial
#Answer is the class which contains response given by each user, it also contain traits to know which person answered the questions.
#llm is inteligent enough to give unique response each time based on the given system and user prompt, we should not worry about it.
class Answer(BaseModel):
    """Elements of JSON structure that is extracted from User Prompts. """

    type: str = Field(description="The type command extracted from User Prompt.")
    andSelect: list = Field(description="The list of conditions based on select and where statement extracted from User Prompt.")
    transformColumn: str = Field(description="The column that needs to be transformed extracted from User Prompt.")
    transformOperation: str = Field(description="The transformation operation needs to be done on column given in User Prompt.")
    transformValue: str = Field(description="The value that needs to be transformed.")
    mode: str = Field(description="Mode of the user prompt")
    selectExpr: str = Field(description="Count of values in select statement that needs to be transformed")
    headerName: str = Field(description="Column name given in User Prompt")
    
    
    
#list of answers, the formatted list that the llm model will generate which contains dictionaries of Answer class
#llm model will make sure that each dictionary in the list is unique from each other as it will treat the list like list of human responses.
# class Answers(BaseModel):
#     """Human-like survey answers to tell user."""

#     answer: List[Answer]

In [11]:
openai_functions = [convert_pydantic_to_openai_function(Answer)]

In [12]:
openai_functions

[{'name': 'Answer',
  'description': 'Elements of JSON structure that is extracted from User Prompts. ',
  'parameters': {'type': 'object',
   'properties': {'type': {'description': 'The type command extracted from User Prompt.',
     'type': 'string'},
    'andSelect': {'description': 'The list of conditions based on select and where statement extracted from User Prompt.',
     'type': 'array',
     'items': {}},
    'transformColumn': {'description': 'The column that needs to be transformed extracted from User Prompt.',
     'type': 'string'},
    'transformOperation': {'description': 'The transformation operation needs to be done on column given in User Prompt.',
     'type': 'string'},
    'transformValue': {'description': 'The value that needs to be transformed.',
     'type': 'string'},
    'mode': {'description': 'Mode of the user prompt', 'type': 'string'},
    'selectExpr': {'description': 'Count of values in select statement that needs to be transformed',
     'type': 'string

In [53]:
system_message = """
You are a JSON extracting bot from the given User prompt. You need to return JSON by extracting useful elements from the User Prompt.
User prompt generally is statements of SQL commands.
Here are few examples of User prompt and the JSON format that needs to be extracted from the User prompt:

User Prompt: Select rows where column 9 contains USD. Then, prepend 'ID-' to the beginning of column 2.
Answer:
	"type": "SelectAndTransformCommand",
	"andSelect": [
		"9 containsIgnoreCase USD"
	],
	"transformColumn": "2",
	"transformOperation": "prepend",
	"transformValue": "ID-",
	"mode": "Modify"

User Prompt: change column 9 to this or last working date.
Answer:
	"type": "SelectAndTransformCommand",
	"transformColumn": "9",
	"transformOperation": "thisOrLastWorkingDate",
	"mode": "Modify"

User Prompt: Remove all instances of "0.000000" from within Column 1
Answer:
	"type": "SelectAndTransformCommand",
 	"selectExpr": "ALL",
	"transformColumn": "1",
	"transformOperation": "removeAllX",
	"transformValue": "0.000000",
	"mode": "Modify"

User Prompt: Where Column 78 is not blank, keep the first three characters
Answer:
	"type": "SelectAndTransformCommand",
	"andSelect": [
		"78 notEquals "
	],
	"transformColumn": "78",
	"transformOperation": "keepFirstXChars",
	"transformValue": "3",
	"mode": "Modify"

User Prompt: Round all numerical information within column 9 to two decimal places
Answer:
	"type": "SelectAndTransformCommand",
	"selectExpr": "ALL",
	"transformColumn": "9",
	"transformOperation": "rounding",
	"transformValue": "2",
	"mode": "Modify"

User Prompt: Add a new column "One" with value 1 for all rows
Answer:
	"type": "AddColumnCommand",
	"headerName": "One",
 	"selectExpr": "ALL",
	"value": "1"

"""

In [54]:
#bind openai_functions to llm for function calling
llm = ChatOpenAI(
    temperature=0,
    model_name=model_name
).bind(functions=openai_functions)
# llm = ChatOpenAI(
#     temperature=0,
#     model_name=model_name
# )

In [55]:
#chat template
prompt = ChatPromptTemplate.from_messages(
    [("system", system_message), ("user", "{input}")]
)

In [56]:
#chain using prompt, llm and parser
#it uses the prompt with llm and generate the answer based on the parser

# chain = prompt | llm | parser

chain = prompt | llm 

In [58]:
def output_parser(response):
    try:
        return eval(response.additional_kwargs['function_call']['arguments'])
    except Exception as e:
        try:
            return json_repair.loads(response.additional_kwargs['function_call']['arguments'])
        except Exception as e:
            return None

In [61]:
user_prompt = 'Where Column 77 is not equal to " ", then append "Sell" to Column 77'

input_message = f"Generate JSON format from given User Prompt.\n"
input_message += f'This is User Prompt: {user_prompt}\n'
res = chain.invoke({"input": input_message})

In [62]:
output_parser(res)

{'type': 'SelectAndTransformCommand',
 'andSelect': ['77 notEquals " "'],
 'transformColumn': '77',
 'transformOperation': 'append',
 'transformValue': 'Sell',
 'mode': 'Modify'}